In [1]:
%run '../scripts/joplin-to-parquet.py'

In [2]:
import warnings
warnings.filterwarnings('ignore') # i was getting some warnings about missing glyphs in font

In [3]:
def add_hour_of_week(row):
    import pytz
    offset = {
        "N. Virginia":  "EST",
        "Ireland":  "GMT",
        "Global":  "GMT",
        "N. California":  "US/Pacific",
        "Sydney":  "Australia/Sydney",
        "Oregon":  "US/Pacific",
        "GovCloud":  "US/Pacific",
        "London":  "Europe/London",
        "Ohio":  "EST",
        "Sao Paulo":  "America/Sao_Paulo",
        "Tokyo":  "Asia/Tokyo",
        "Mumbai":  "Asia/Kolkata",
        "Singapore":  "Asia/Singapore",
        "Seoul":  "Asia/Seoul",
        "Frankfurt":  "Europe/Berlin",
        "Paris":  "Europe/Paris",
        "US-West":  "US/Pacific",
        "East US":  "EST",
        "global":  "GMT",
        "RCA – Resources using IPv4 addressing – West and South India":  "Asia/Kolkata",
        "West Europe":  "Europe/Amsterdam",
        "West US":  "US/Pacific",
        "Australia East":  "Australia/Sydney",  # based on https://azure.microsoft.com/en-us/global-infrastructure/regions/
        "North Central US":  "America/Chicago",  # based on https://azure.microsoft.com/en-us/global-infrastructure/regions/
        "North Europe and West Europe":  "Europe/Berlin",
        "UK West":  "Europe/London",
        "South Central US":  "US/Central",
        "RCA – Storage – West US":  "US/Pacific",
        "West India and South India":  "Asia/Kolkata",
        "Latency between North Europe and North America":  "GMT",
        "France Central":  "Europe/Paris",
        "East Asia":  "Asia/Hong_Kong",  # based on https://azure.microsoft.com/en-us/global-infrastructure/regions/
        "Australia Southeast":  "Australia/Melbourne",  # based on https://azure.microsoft.com/en-us/global-infrastructure/regions/
        "Korea South":  "Asia/Seoul",
        "Southeast Asia":  "Asia/Singapore",
        "West Europe and North Europe":  "Europe/Berlin",
        "Latency and Slow I/O issues in East US":  "EST",
        "Networking in West US":  "US/Pacific",
        "UK South":  "Europe/London",
        "West US 2":  "US/Central",
        "East US and West US":  "US/Central",
        "UK South and UK West":  "Europe/London",
        "North Europe":  "Europe/Berlin",
        "UK South/UK West":  "Europe/London",
        "West Central US":  "US/Central",
        "West Europe | Mitigated":  "Europe/Amsterdam",
        "Data Processing in East US":  "EST",
        "Australia East/Southeast":  "Australia/Melbourne",
        "Canada Central":  "Canada/Central",
        "Japan East":  "Japan",
        "Multiple Azure Services impacted in West Europe":  "Europe/Amsterdam",
        "Service availability issue in North Europe":  "Europe/Berlin",
        "Service Availability Issue in North Europe":  "Europe/Berlin",
        "South East Asia":  "Asia/Singapore",
    }
    try:
        timezone_diff = pytz.timezone(offset[row['location']])
        localized_event_start_time = row['event_start_time'].tz_convert(timezone_diff)
        weekday_int = localized_event_start_time.dayofweek
        hour = localized_event_start_time.hour
        row['hour_of_week'] = (weekday_int+1)*24+hour
    except KeyError:
        # TODO: had some issue with Canada
        pass
    return row

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_parquet("../data/classified.parquet")
original = pd.read_parquet("../data/outages.parquet")
dfj = df.join(original)

In [5]:
dfj['duration_min'] = (dfj.event_end_time - dfj.event_start_time)/60.0
dfj["event_start_time"] = pd.to_datetime(dfj["event_start_time"], unit="s", utc=True)
dfj["event_end_time"] = pd.to_datetime(dfj["event_end_time"], unit="s", utc=True)
dfj = dfj.apply(add_hour_of_week, axis='columns')
dfj.drop(labels=['event_start_time', 'event_end_time'], axis=1, inplace=True)

In [6]:
def clean_emptys(cell):
    from numpy import ndarray
    if type(cell) == ndarray and len(cell) == 0:
        return ['not provided']
    elif type(cell) == str and not bool(cell):
        return 'not provided'
    
    return cell

dfj = dfj.applymap(clean_emptys)

In [7]:
def deduplicate(df):
    df['cutdesc'] = df.description.apply(lambda x: x[25:])
    df = df.drop_duplicates(['cutdesc']).drop(labels=['cutdesc'], axis=1)
    return df

dfj = deduplicate(dfj)

In [8]:
dfj.to_parquet("../data/preprocessed.parquet")